## Helper Functions

In [1]:
# # =========================
# # Sentinel-2 + Cloud Probability (ROI-first)
# # =========================

# regions = {
#     "spain": ee.Geometry.Rectangle([-5.1, 41.9, -4.9, 42.1]),
#     "france": ee.Geometry.Rectangle([1.95, 48.05, 2.05, 48.20]),
#     "austria": ee.Geometry.Rectangle([16.70, 48.10, 16.80, 48.20]),
#     "romania": ee.Geometry.Rectangle([27.25, 44.20, 27.35, 44.28]),
# }

# roi = regions["spain"].buffer(2000)


# cloudprob = (ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")
#             .filterBounds(roi)
#             .filterDate(date_start, date_end))

# dataSE = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
#           .filterBounds(roi)
#           .filterDate(date_start, date_end)
#           .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover)))

# # Join by system:index; keep first cloud prob img as property "Cloud"
# idJoin    = ee.Filter.equals(leftField='system:index', rightField='system:index')
# innerJoin = ee.Join.saveFirst('Cloud')
# dataSE = ee.ImageCollection(innerJoin.apply(primary=dataSE, secondary=cloudprob, condition=idJoin))

# # Preprocess and immediately CLIP to ROI
# dataSE = (preproS2(dataSE, cloud_prob)
#           .select(bands)
#           .map(lambda img: ee.Image(img).clip(roi)))

# # Spectral indices (on already-clipped images)
# s2Idx = vegetation_indices(dataSE)

# # Monthly S2 (median for months 3..10) toBands
# months = ee.List.sequence(3, 10)
# S2_Monthly = generateS2month(months, s2Idx, year).toBands()

# # Yearly S2 (5/50/98) with renamed bands
# S2_Yearly = ee.Image(s2Idx.reduce(ee.Reducer.percentile([5, 50, 98])))
# S2_Yearly = S2_Yearly.rename(newNames(S2_Yearly, year)).clip(roi)

# # =========================
# # Sentinel-1 (ROI-first)
# # =========================
# radar = (ee.ImageCollection("COPERNICUS/S1_GRD")
#         .filterBounds(roi)
#         .filterDate('2022-01-01', '2023-01-01')
#         .filter(ee.Filter.eq('instrumentMode', 'IW'))
#         .filter(ee.Filter.listContains('transmitterReceiverPolarisation','VV'))
#         .select(['VV','VH']))

# radar = preproS1(radar, roi)
# radar = s1_features(radar)

# # Monthly S1 + Yearly S1
# S1_Monthly = generateS1month(radar, '2022-01-01', 12, 1, 'month').toBands()
# S1_Yearly  = radar.reduce(ee.Reducer.percentile([5, 50, 98])).clip(roi)

# # =========================
# # LST + DEM (ROI-first)
# # =========================
# lst = (ee.ImageCollection('MODIS/061/MOD21C3')
#       .filterBounds(roi)
#       .filterDate('2022-01-01', '2022-12-31')
#       .select('LST_Day'))
# lst = clipic(lst, roi).toBands()

# dem = ee.Image("projects/sat-io/open-datasets/ASTER/GDEM").clip(roi)

# # =========================
# # Stack & Classify
# # =========================
# data = ee.Image.cat([
#     S2_Monthly,
#     S2_Yearly,
#     S1_Monthly,
#     S1_Yearly,
#     lst,
#     dem
# ]).clip(roi)

# final_data = data.select(feat_select)
# class_map  = final_data.classify(model).rename('class')

# # # Map post-processing / code remap to EU classes and styling
# class_map_remap = rename_classes(class_map)
# styled_rgb = Visualization(class_map_remap)


In [2]:
# ===== Earth Engine helpers (Python port of your JS code) =====
# Requires: pip install earthengine-api (and optionally geemap for viz in Colab)
# Auth (one-time):   import ee; ee.Authenticate()
# Init (each session): ee.Initialize()
# --------------------------- S2 PREPROCESS ---------------------------
import ee

ee.Authenticate()
ee.Initialize(project='time-series-462612')


def preproS2(ic, cloud_prob):
    """
    Args:
      ic: ee.ImageCollection of Sentinel-2 images where each image has an Image
          under property 'Cloud' with band 'probability', and has SCL band.
      cloud_prob: numeric threshold for Cloud probability (0..100)
    Returns:
      ee.ImageCollection with optical bands (B.*) masked by clouds & land cover.
    """
    def _map(img):
        img = ee.Image(img)
        cloud_img = ee.Image(img.get('Cloud'))
        mask_cloud = cloud_img.select('probability').lte(cloud_prob)
        mask_land = img.select('SCL').gt(1).And(img.select('SCL').lt(9))
        mask = mask_cloud.And(mask_land)
        return (img.select('B.*')
                   .updateMask(mask)
                   .copyProperties(img, ['system:time_start']))
    return ee.ImageCollection(ic.map(_map))

# --------------------------- S2 INDICES ---------------------------

def _ndvi(image):
    ndvi_val = image.normalizedDifference(['B8', 'B4']).rename('NDVI')
    return image.addBands(ndvi_val)

def _srnirr(image):
    srnirr_val = image.expression(
        '(B8/B4)', {'B4': image.select('B4'), 'B8': image.select('B8')}
    ).rename('SRNIRR')
    return image.addBands(srnirr_val)

def _direswir(image):
    direswir_val = image.expression(
        '(B4 - B11)',
        {'B11': image.select('B11').multiply(0.0001),
         'B4':  image.select('B4').multiply(0.0001)}
    ).rename('DIRESWIR')
    return image.addBands(direswir_val)

def _lcci(image):
    lcci_val = image.expression(
        '(B7/B5)', {'B7': image.select('B7'), 'B5': image.select('B5')}
    ).rename('LCCI')
    return image.addBands(lcci_val)

def _gnvi(image):
    gndvi_val = image.normalizedDifference(['B8', 'B3']).rename('gndvi')
    return image.addBands(gndvi_val)

def _evi(image):
    evi_val = image.expression(
        '2.4 * ((B8 - B4) / (B8 + B4 + 1))',
        {'B8': image.select('B8').multiply(0.0001),
         'B4': image.select('B4').multiply(0.0001)}
    ).rename('EVI2')
    return image.addBands(evi_val)

def _lai(image):
    # LAI = 3.618 * EVI - 0.118, with EVI = 2.5 * (B8 - B4) / (B8 + 6B4 - 7.5B2 + 1)
    lai_val = image.expression(
        '3.618 * (2.5 * ((B8 - B4) / (B8 + 6 * B4 - 7.5 * B2 + 1))) - 0.118',
        {'B8': image.select('B8').multiply(0.0001),
         'B4': image.select('B4').multiply(0.0001),
         'B2': image.select('B2').multiply(0.0001)}
    ).rename('LAI')
    return image.addBands(lai_val)

def _laig(image):
    # LAIg = 5.405 * ((B8A - B5)/(B8A + B5)) - 0.114
    laig_val = image.expression(
        '5.405 * ((B8A - B5) / (B8A + B5)) - 0.114',
        {'B8A': image.select('B8A').multiply(0.0001),
         'B5':  image.select('B5').multiply(0.0001)}
    ).rename('LAIg')
    return image.addBands(laig_val)

def _msavi(image):
    msavi_val = image.expression(
        '0.5 * (2 * B8 + 1 - sqrt((2 * B8 + 1) ** 2 - 8 * (B8 - B4)))',
        {'B8': image.select('B8').multiply(0.0001),
         'B4': image.select('B4').multiply(0.0001)}
    ).rename('MSAVI')
    return image.addBands(msavi_val)

def _savi(image):
    savi_val = image.expression(
        '(1.5 * ((B8 - B4) / (B8 + B4 + 0.5)))',
        {'B8': image.select('B8').multiply(0.0001),
         'B4': image.select('B4').multiply(0.0001)}
    ).rename('SAVI')
    return image.addBands(savi_val)

def _bsi(image):
    bsi_val = image.expression(
        '((B11 + B4) - (B8 + B2)) / ((B11 + B4) + (B8 + B2))',
        {'B11': image.select('B11'),
         'B4':  image.select('B4'),
         'B8':  image.select('B8'),
         'B2':  image.select('B2')}
    ).rename('BSI')
    return image.addBands(bsi_val)

def _ndti(image):
    ndti_val = image.normalizedDifference(['B11', 'B12']).rename('NDTI')
    return image.addBands(ndti_val)

def _ndbi(image):
    ndbi_val = image.normalizedDifference(['B11', 'B8']).rename('NDBI')
    return image.addBands(ndbi_val)

def _blfei(image):
    blfei_val = image.expression(
        '((B3 + B4 + B12) / 3 - B11) / (((B3 + B4 + B12) / 3) + B11)',
        {'B3':  image.select('B3'),
         'B4':  image.select('B4'),
         'B12': image.select('B12'),
         'B11': image.select('B11')}
    ).rename('BLFEI')
    return image.addBands(blfei_val)

def _mndwi(image):
    mndwi_val = image.normalizedDifference(['B3', 'B11']).rename('MNDWI')
    return image.addBands(mndwi_val)

def _ndwi(image):
    ndwi_val = image.normalizedDifference(['B8', 'B11']).rename('NDWI')
    return image.addBands(ndwi_val)

def vegetation_indices(ic):
    """
    Apply all 16 indices and return the ImageCollection with added bands.
    """
    for f in (_ndvi, _srnirr, _direswir, _lcci, _gnvi, _evi, _lai, _laig,
              _msavi, _savi, _bsi, _ndti, _ndbi, _blfei, _mndwi, _ndwi):
        ic = ic.map(f)
    return ic

# --------------------------- S2 MONTHLY AGG ---------------------------

def generateS2month(months, data, yr):
    """
    Args:
      months: ee.List of month integers (1..12)
      data: ee.ImageCollection (e.g., preprocessed S2)
      yr: int or string year label to prefix bands with
    Returns:
      ee.ImageCollection with monthly medians (Jan-Mar combined if m<=3),
      bands renamed to '{yr}_{band}'.
    """
    def _per_month(m):
        m = ee.Number(m)
        img = ee.Image(ee.Algorithms.If(
            m.lte(3),
            data.filter(ee.Filter.calendarRange(1, 3, 'month')).median()
                .set({'month': m, 'year': yr}),
            data.filter(ee.Filter.calendarRange(m, m, 'month')).median()
                .set({'month': m, 'year': yr})
        ))
        prefix = ee.String(str(yr) + '_')
        newnames = img.bandNames().map(lambda b: prefix.cat(ee.String(b)))
        return img.rename(newnames)
    return ee.ImageCollection.fromImages(months.map(_per_month))

def newNames(data, yr):
    """
    Returns ee.List of new band names with '{yr}_' prefix.
    """
    return data.bandNames().map(lambda bd: ee.String(str(yr) + '_').cat(ee.String(bd)))

# --------------------------- S1 PREPROCESS & FEATURES ---------------------------

def preproS1(collection, geo):
    """
    Convert backscatter from dB to linear, speckle reduce (median 30m), clip to geo.
    Preserves system:time_* properties.
    """
    def _map(img):
        sigma0 = ee.Image(10).pow(ee.Image(img).divide(10.0))
        speckle = sigma0.focal_median(30, 'circle', 'meters')
        img_clip = speckle.clip(geo)
        return img_clip.copyProperties(img, ['system:time_start', 'system:time_end'])
    return collection.map(_map)

def s1_features(collection):
    """
    Add RVI, VV/VH, VV-VH, DPSVIm, NDPI bands.
    Assumes input images already have bands 'VV' and 'VH' (linear scale).
    """
    def _feat(image):
        vv = image.select('VV')
        vh = image.select('VH')
        rvi = image.expression('4 * VH / (VV + VH)', {'VV': vv, 'VH': vh}).rename('RVI')
        ratio = vv.divide(vh).rename('VV/VH')
        diff  = vv.subtract(vh).rename('VV-VH')
        dpsv  = image.expression('(VV*VV + VV*VH)/1.414213562', {'VV': vv, 'VH': vh}).rename('DPSVIm')
        ndpi  = image.expression('(VV - VH) / (VV + VH)', {'VV': vv, 'VH': vh}).rename('NDPI')
        return image.addBands([rvi, ratio, diff, dpsv, ndpi])
    return collection.map(_feat)

def generateS1month(collection, start, count, interval, units):
    """
    Slice collection into consecutive windows and take median per window.
    Args:
      collection: ee.ImageCollection (e.g., S1 processed)
      start: ISO date string or ee.Date
      count: number of windows
      interval: window length (int)
      units: 'day'|'week'|'month'|'year'
    """
    origin = ee.Date(start)
    idx = ee.List.sequence(0, ee.Number(count).subtract(1))
    def _window(i):
        i = ee.Number(i)
        t0 = origin.advance(i.multiply(interval), units)
        t1 = origin.advance(i.add(1).multiply(interval), units)
        return (collection.filterDate(t0, t1).median()
                .set({'system:time_start': t0.millis(),
                      'system:time_end':   t1.millis()}))
    return ee.ImageCollection(idx.map(_window))

# --------------------------- GENERIC HELPERS ---------------------------

def clipic(collection, geo):
    return collection.map(lambda image: ee.Image(image).clip(geo.bounds()))

def rename_classes(img):
    """
    Remap class codes to the EU-CropMap codes using chained .where to match your JS.
    """
    img = ee.Image(img)
    mapping = [
        (0,100),(1,211),(2,212),(3,213),(4,214),(5,215),(6,216),(7,217),(8,218),(9,219),
        (10,221),(11,222),(12,223),(13,230),(14,231),(15,232),(16,233),(17,240),
        (18,250),(19,290),(20,300),(21,500),(22,600),(23,700),(24,800)
    ]
    for old, new in mapping:
        img = img.where(img.eq(old), new)
    return img

def postMap(img):
    """
    Small-patch removal + modal filter smoothing (mirrors your focal_mode + connectedPixelCount).
    """
    img = ee.Image(img)
    patchsize = img.connectedPixelCount(30, eightConnected=False)
    filtered = img.focal_mode(radius=10, kernelType='square', units='pixels')
    return img.where(patchsize.lt(20), filtered)

# --------------------------- SLD & VIZ / LEGEND ---------------------------

sld_intervals = (
  '<RasterSymbolizer>'
  '<ColorMap type="intervals" extended="false">'
    '<ColorMapEntry color="#ff130f" quantity="100" label="Artificial"/>'
    '<ColorMapEntry color="#a57000" quantity="211" label="Common wheat"/>'
    '<ColorMapEntry color="#896054" quantity="212" label="Durum wheat"/>'
    '<ColorMapEntry color="#e2007c" quantity="213" label="Barley"/>'
    '<ColorMapEntry color="#aa007c" quantity="214" label="Rye"/>'
    '<ColorMapEntry color="#a05989" quantity="215" label="Oats"/>'
    '<ColorMapEntry color="#ffd300" quantity="216" label="Maize"/>'
    '<ColorMapEntry color="#00a8e2" quantity="217" label="Rice"/>'
    '<ColorMapEntry color="#d69ebc" quantity="218" label="Triticale"/>'
    '<ColorMapEntry color="#d69ebc" quantity="219" label="Other cereals"/>'
    '<ColorMapEntry color="#dda50a" quantity="221" label="Potatoes"/>'
    '<ColorMapEntry color="#a800e2" quantity="222" label="Sugar beet"/>'
    '<ColorMapEntry color="#00af49" quantity="223" label="Other roots crops"/>'
    '<ColorMapEntry color="#00af49" quantity="230" label="Other non permanent industrial crops"/>'
    '<ColorMapEntry color="#ffff00" quantity="231" label="Sunflower"/>'
    '<ColorMapEntry color="#d1ff00" quantity="232" label="Rape and turnip rape"/>'
    '<ColorMapEntry color="#267000" quantity="233" label="Soya"/>'
    '<ColorMapEntry color="#f2a377" quantity="240" label="Dry pulses, vegetables and flowers"/>'
    '<ColorMapEntry color="#e8bfff" quantity="250" label="Other fodder crops"/>'
    '<ColorMapEntry color="#000000" quantity="290" label="Bare arable land"/>'
    '<ColorMapEntry color="#93cc93" quantity="300" label="Woodland and shrubland type of vegetation"/>'
    '<ColorMapEntry color="#e8ffbf" quantity="500" label="Grassland"/>'
    '<ColorMapEntry color="#a89e7f" quantity="600" label="Bare land/lichens moss"/>'
    '<ColorMapEntry color="#0793de" quantity="700" label="Water"/>'
    '<ColorMapEntry color="#7cafaf" quantity="800" label="Wetlands"/>'
  '</ColorMap>'
  '</RasterSymbolizer>'
)

def Visualization(img):
    """
    Apply SLD intervals styling (server-side). Returns an RGB visualization image.
    """
    return ee.Image(img).sldStyle(sld_intervals)

def legend_items_from_sld():
    """
    Parse the SLD string and return a client-side list of dicts: [{'color':'#...', 'label':'...'}, ...]
    Useful to render a legend in geemap/folium, Streamlit, etc.
    """
    import re
    pattern = re.compile(r'<ColorMapEntry color="([^"]+)" quantity="[^"]+" label="([^"]+)"\/>')
    return [{'color': c, 'label': l} for (c, l) in pattern.findall(sld_intervals)]


## Train

In [3]:
train_p1 = ee.FeatureCollection("projects/ee-babakghassemi9/assets/LU22_S1S2MYA_CentroPolyPointsFull_training75_part_1")
train_p2 = ee.FeatureCollection("projects/ee-babakghassemi9/assets/LU22_S1S2MYA_CentroPolyPointsFull_training75_part_2")
train_p3 = ee.FeatureCollection("projects/ee-babakghassemi9/assets/LU22_S1S2MYA_CentroPolyPointsFull_training75_part_3")
train_p4 = ee.FeatureCollection("projects/ee-babakghassemi9/assets/LU22_S1S2MYA_CentroPolyPointsFull_training75_part_4")

f_name1 = ee.FeatureCollection("projects/ee-babakghassemi9/assets/LU22_S1S2MYA_important_features2")

train_all = (train_p1
             .merge(train_p2)
             .merge(train_p3)
             .merge(train_p4))


In [4]:
import geemap

# remove_list = ['6_VV', '3_VH', '5_VV', '6_DPSVIm', '4_VH', 'VH_p50', 'DPSVIm_p5',
#                '5_DPSVIm', 'VH_p5', '4_DPSVIm', '5_VH', 'RVI_p5', 'VV/VH_p50',
#                'NDPI_p50', '6_VV-VH', '4_VV', '5_VV-VH', '7_VH', '6_VH', '2_VH']


year        = '2022'
date_start  = ee.Date(f'{year}-01-01')
date_end    = ee.Date(f'{year}-12-31')
cloud_cover = 50
cloud_prob  = 75
bands = ['B2','B3','B4','B5','B6','B7','B8','B8A','B11','B12']

feat_select = ee.List(ee.FeatureCollection(f_name1).aggregate_array('0')).distinct()
# feat_select = feat_select.removeAll(ee.List(remove_list))


# =========================
# Classifier (Random Forest)
# =========================
model = ee.Classifier.smileRandomForest(
    numberOfTrees=150,
    variablesPerSplit=None,
    minLeafPopulation=2,
    bagFraction=0.5,
    maxNodes=None,
    seed=0
)



In [5]:
# Look at the properties on the first training feature
print('Training feature properties:')
print(train_all.first().propertyNames().getInfo())


Training feature properties:
['system:index', '7_2022_NDWI', '8_VH', '6_VV-VH', '7_2022_DIRESWIR', '0_2022_LCCI', 'b1', '8_DPSVIm', '2022_B8_p5', '5_2022_NDTI', '4_DPSVIm', '2022_B6_p50', '7_2022_NDVI', '0_DPSVIm', '4_2022_NDBI', '5_NDPI', '4_2022_MSAVI', '2022_LAI_p50', 'Label_lev2_code', '2022_SRNIRR_p5', '8_VV', '2022_10_01_LST_Day', '3_2022_MNDWI', 'DPSVIm_p98', '2022_NDWI_p98', '1_VV-VH', '2022_DIRESWIR_p50', '2022_BLFEI_p98', '3_2022_NDTI', '6_2022_BLFEI', '3_2022_LAI', 'VV-VH_p50', '2_2022_B8A', '9_VH', '5_RVI', '3_2022_NDVI', '7_2022_gndvi', '2022_B7_p5', '2_2022_NDBI', '1_2022_B8A', '4_2022_LCCI', '6_2022_MSAVI', 'NDPI_p50', '2022_NDTI_p98', '3_2022_BSI', '6_VV/VH', '3_2022_NDWI', '9_VV', 'Label_clas', '2022_MNDWI_p98', '11_VV/VH', '2022_EVI2_p5', '2_NDPI', '0_2022_NDBI', '2022_07_01_LST_Day', '6_2022_B2', '6_2022_B3', '6_2022_B4', '6_2022_B5', '6_2022_B6', '6_2022_B7', '6_2022_B8', '7_2022_B11', '1_2022_NDVI', '7_2022_B12', '2022_LCCI_p5', '2022_B11_p98', 'DPSVIm_p50', '2022_

In [6]:
feat_select_updated = feat_select.map(
    lambda s: ee.String(s).replace("2022", "2025")
)

print("Updated:", feat_select_updated.getInfo())

Updated: ['0_2025_B3', '0_2025_B4', '0_2025_B5', '0_2025_BLFEI', '0_2025_LAI', '0_2025_LAIg', '0_2025_LCCI', '0_2025_MSAVI', '0_2025_NDVI', '0_2025_SAVI', '0_2025_SRNIRR', '0_2025_gndvi', '1_2025_B6', '1_2025_B8', '1_2025_BLFEI', '1_2025_DIRESWIR', '1_2025_EVI2', '1_2025_LAI', '1_2025_LAIg', '1_2025_LCCI', '1_2025_MNDWI', '1_2025_MSAVI', '1_2025_NDTI', '1_2025_NDVI', '1_2025_SAVI', '1_2025_SRNIRR', '1_2025_gndvi', '2025_01_01_LST_Day', '2025_02_01_LST_Day', '2025_03_01_LST_Day', '2025_04_01_LST_Day', '2025_05_01_LST_Day', '2025_06_01_LST_Day', '2025_07_01_LST_Day', '2025_08_01_LST_Day', '2025_09_01_LST_Day', '2025_10_01_LST_Day', '2025_11_01_LST_Day', '2025_12_01_LST_Day', '2025_B11_p98', '2025_B12_p50', '2025_B12_p98', '2025_B2_p50', '2025_B3_p50', '2025_B4_p50', '2025_B5_p50', '2025_B6_p98', '2025_B7_p98', '2025_BLFEI_p5', '2025_BLFEI_p50', '2025_BSI_p5', '2025_BSI_p50', '2025_BSI_p98', '2025_DIRESWIR_p50', '2025_EVI2_p5', '2025_EVI2_p98', '2025_LAI_p5', '2025_LAI_p98', '2025_LAIg_p5

In [7]:
model = model.train(
    features = ee.FeatureCollection(train_all).filter(ee.Filter.eq('Lbl_cls_major', 1)),
    classProperty = 'Label_clas',
    inputProperties = feat_select
)

## Inference

### Inference

In [ ]:
def build_s2_features(roi,
                      date_start,
                      date_end,
                      year,
                      bands,
                      cloud_cover,
                      cloud_prob):
    """
    Return:
      S2_Monthly 208 = 8 * 26
      S2_Yearly 78
    stacks for a given ROI/year.
    """
    # Cloud probability and S2 base collection
    cloudprob = (ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")
                 .filterBounds(roi)
                 .filterDate(date_start, date_end))

    dataSE = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
              .filterBounds(roi)
              .filterDate(date_start, date_end)
              .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover)))

    # Join by system:index; keep first cloud prob img as property "Cloud"
    idJoin    = ee.Filter.equals(leftField='system:index', rightField='system:index')
    innerJoin = ee.Join.saveFirst('Cloud')
    dataSE    = ee.ImageCollection(innerJoin.apply(primary=dataSE,
                                                   secondary=cloudprob,
                                                   condition=idJoin))

    # Preprocess and immediately CLIP to ROI
    dataSE_processed = (preproS2(dataSE, cloud_prob)
              .select(bands)
              .map(lambda img: ee.Image(img).clip(roi)))

    # Spectral indices (already clipped)
    s2Idx = vegetation_indices(dataSE_processed)

    # Monthly S2 (median for months 3..10) -> toBands
    months     = ee.List.sequence(3, 10)
    S2_Monthly = generateS2month(months, s2Idx, year).toBands()

    # Yearly S2 (5/50/98) with renamed bands
    S2_Yearly = ee.Image(s2Idx.reduce(ee.Reducer.percentile([5, 50, 98])))
    S2_Yearly = S2_Yearly.rename(newNames(S2_Yearly, year)).clip(roi)

    print("S2_Monthly", len(S2_Monthly.bandNames().getInfo()))
    print("S2_Yearly", len(S2_Yearly.bandNames().getInfo()))

    return S2_Monthly, S2_Yearly


def build_s1_features(roi,
                      start_date='2022-01-01',
                      end_date='2023-01-01'):
    """
    Return:
      S1_Monthly 84 = 7 * 12
      S1_Yearly 21
    stacks for Sentinel-1.
    """

    radar = (ee.ImageCollection("COPERNICUS/S1_GRD")
        .filterBounds(roi)
        .filterDate(start_date, end_date)
        .filter(ee.Filter.eq('instrumentMode', 'IW'))
        .filter(ee.Filter.listContains('transmitterReceiverPolarisation','VV'))
        .select(['VV','VH']))

    radar = preproS1(radar, roi)
    radar = s1_features(radar)

    # Monthly S1 + Yearly S1
    S1_Monthly = generateS1month(radar, start_date, 12, 1, 'month').toBands()
    S1_Yearly  = radar.reduce(ee.Reducer.percentile([5, 50, 98])).clip(roi)

    print("S1_Monthly", len(S1_Monthly.bandNames().getInfo()))
    print("S1_Yearly", len(S1_Yearly.bandNames().getInfo()))


    return S1_Monthly, S1_Yearly


def build_lst_features(roi,
                       year=2022):
    """Return LST stack (monthly) for given year."""
    start = f'{year}-01-01'
    end   = f'{year}-12-31'

    lst = (ee.ImageCollection('MODIS/061/MOD21C3')
           .filterBounds(roi)
           .filterDate(start, end)
           .select('LST_Day'))

    # Clip and convert toBands (e.g. monthly, if clipic does temporal grouping)
    lst = clipic(lst, roi).toBands()
    return lst


def build_dem(roi):
    """Return DEM clipped to ROI."""
    dem = ee.Image("projects/sat-io/open-datasets/ASTER/GDEM").clip(roi)
    return dem



In [ ]:
regions = {
    "spain": ee.Geometry.Rectangle([-5.1, 41.9, -4.9, 42.1]),
    "france": ee.Geometry.Rectangle([1.95, 48.05, 2.05, 48.20]),
    "austria": ee.Geometry.Rectangle([16.70, 48.10, 16.80, 48.20]),
    "romania": ee.Geometry.Rectangle([27.25, 44.20, 27.35, 44.28]),
}

roi = regions["romania"].buffer(2000)


S2_Monthly, S2_Yearly = build_s2_features(
    roi=roi,
    date_start=date_start,
    date_end=date_end,
    year=year,
    bands=bands,
    cloud_cover=cloud_cover,
    cloud_prob=cloud_prob
)

S1_Monthly, S1_Yearly = build_s1_features(
    roi=roi
    # start_date=date_start,
    # end_date=date_end
)

lst = build_lst_features(roi=roi, year=year)
dem = build_dem(roi=roi)

data = ee.Image.cat([
    S2_Monthly,
    S2_Yearly,
    S1_Monthly,
    S1_Yearly,
    lst,
    dem
]).clip(roi)


final_data = data.select(feat_select_updated)
class_map  = final_data.classify(model).rename('class')

# # Map post-processing / code remap to EU classes and styling
class_map_remap = rename_classes(class_map)
styled_rgb = Visualization(class_map_remap)

S2_Monthly 208
S2_Yearly 78
S1_Monthly 84
S1_Yearly 21


### Visualization

In [ ]:
dataSE = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
          .filterBounds(roi)
          .filterDate(date_start, date_end)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover)))

m = geemap.Map(center=[48, 67], zoom=4)

truecolor = (dataSE.median()
             .select(['B4','B3','B2'])
             .visualize(min=0, max=3000))
m.add_layer(truecolor, {}, 'S2 True Color (median)', opacity=0.6)

m.add_layer(styled_rgb, {}, 'EU CropMap 2022 (classified)', opacity=1.0)

# Build legend from SLD
items = legend_items_from_sld()   # [{'color':'#hex', 'label':'...'}, ...]
legend_dict = {it['label']: it['color'] for it in items}
m.add_legend(title='EU Crop Classes', legend_dict=legend_dict, position='bottomright')
m.centerObject(roi, 12)
m

EEException: Image.select: Band pattern '2025_11_01_LST_Day' did not match any bands. Available bands: [0_2025_B2, 0_2025_B3, 0_2025_B4, 0_2025_B5, 0_2025_B6, 0_2025_B7, 0_2025_B8, 0_2025_B8A, 0_2025_B11, 0_2025_B12, 0_2025_NDVI, 0_2025_SRNIRR, 0_2025_DIRESWIR, 0_2025_LCCI, 0_2025_gndvi, 0_2025_EVI2, 0_2025_LAI, 0_2025_LAIg, 0_2025_MSAVI, 0_2025_SAVI, 0_2025_BSI, 0_2025_NDTI, 0_2025_NDBI, 0_2025_BLFEI, 0_2025_MNDWI, 0_2025_NDWI, 1_2025_B2, 1_2025_B3, 1_2025_B4, 1_2025_B5, 1_2025_B6, 1_2025_B7, 1_2025_B8, 1_2025_B8A, 1_2025_B11, 1_2025_B12, 1_2025_NDVI, 1_2025_SRNIRR, 1_2025_DIRESWIR, 1_2025_LCCI, 1_2025_gndvi, 1_2025_EVI2, 1_2025_LAI, 1_2025_LAIg, 1_2025_MSAVI, 1_2025_SAVI, 1_2025_BSI, 1_2025_NDTI, 1_2025_NDBI, 1_2025_BLFEI, 1_2025_MNDWI, 1_2025_NDWI, 2_2025_B2, 2_2025_B3, 2_2025_B4, 2_2025_B5, 2_2025_B6, 2_2025_B7, 2_2025_B8, 2_2025_B8A, 2_2025_B11, 2_2025_B12, 2_2025_NDVI, 2_2025_SRNIRR, 2_2025_DIRESWIR, 2_2025_LCCI, 2_2025_gndvi, 2_2025_EVI2, 2_2025_LAI, 2_2025_LAIg, 2_2025_MSAVI, 2

### Functionized


## Astana Fields Inference

### Feature Selection Logic

---



In [ ]:
# =========================
# Sentinel-2 + Cloud Probability (ROI-first)
# =========================

roi = regions["austria"].buffer(2000)
# roi = rect.buffer(2000)


S2_INPUT_BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

cloudprob = (ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")
             .filterBounds(roi)
             .filterDate(date_start, date_end))

dataSE = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
          .filterBounds(roi)
          .filterDate(date_start, date_end)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover)))

# Join by system:index; keep first cloud prob img as property "Cloud"
idJoin    = ee.Filter.equals(leftField='system:index', rightField='system:index')
innerJoin = ee.Join.saveFirst('Cloud')
dataSE = ee.ImageCollection(innerJoin.apply(primary=dataSE, secondary=cloudprob, condition=idJoin))

# Preprocess and immediately CLIP to ROI
dataSE = (preproS2(dataSE, cloud_prob)
          .select(S2_INPUT_BANDS)
          .map(lambda img: ee.Image(img).clip(roi)))

# Spectral indices (on already-clipped images)
s2Idx = vegetation_indices(dataSE)

# Monthly S2 (median for months 3..10) toBands
months = ee.List.sequence(3, 10)
S2_Monthly = generateS2month(months, s2Idx, year).toBands()

# Yearly S2 (5/50/98) with renamed bands
S2_Yearly = ee.Image(s2Idx.reduce(ee.Reducer.percentile([5, 50, 98])))
S2_Yearly = S2_Yearly.rename(newNames(S2_Yearly, year)).clip(roi)

bands = S2_Monthly.bandNames()
print(len(bands.getInfo()))

bands = S2_Yearly.bandNames()
print(len(bands.getInfo()))


208
78


In [ ]:
# =========================
# Sentinel-1 (ROI-first)
# =========================
radar = (ee.ImageCollection("COPERNICUS/S1_GRD")
         .filterBounds(roi)
         .filterDate('2022-01-01', '2023-01-01')
         .filter(ee.Filter.eq('instrumentMode', 'IW'))
         .filter(ee.Filter.listContains('transmitterReceiverPolarisation','VV'))
         .select(['VV','VH']))

radar = preproS1(radar, roi)
radar = s1_features(radar)

# Monthly S1 + Yearly S1
S1_Monthly = generateS1month(radar, '2022-01-01', 12, 1, 'month').toBands()
S1_Yearly  = radar.reduce(ee.Reducer.percentile([5, 50, 98])).clip(roi)

bands = S1_Monthly.bandNames()
print(len(bands.getInfo()))

bands = S1_Yearly.bandNames()
print(len(bands.getInfo()))



84
21


In [ ]:
feat_list = set(feat_select.getInfo())

S1_Monthly_list = set(S1_Monthly.bandNames().getInfo())
S1_Yearly_list = set(S1_Yearly.bandNames().getInfo())
S2_Monthly_list = set(S2_Monthly.bandNames().getInfo())
S2_Yearly_list = set(S2_Yearly.bandNames().getInfo())

print(list(S1_Monthly_list & feat_list), "\n", len(list(S1_Monthly_list & feat_list)))
print(list(S1_Yearly_list & feat_list), "\n", len(list(S1_Yearly_list & feat_list)))
print(list(S2_Monthly_list & feat_list), "\n", len(list(S2_Monthly_list & feat_list)))
print(list(S2_Yearly_list & feat_list), "\n", len(list(S2_Yearly_list & feat_list)))


# how did they select features?


['5_DPSVIm', '3_VH', '4_VH', '6_VV', '5_VH', '6_VH', '5_VV-VH', '7_VH', '4_DPSVIm', '6_DPSVIm', '6_VV-VH', '4_VV', '5_VV', '2_VH'] 
 14
['DPSVIm_p5', 'VH_p50', 'VH_p5', 'NDPI_p50', 'RVI_p5', 'VV/VH_p50'] 
 6
[] 
 0
[] 
 0


In [ ]:
# =========================
# LST + DEM (ROI-first)
# =========================
lst = (ee.ImageCollection('MODIS/061/MOD21C3')
       .filterBounds(roi)
       .filterDate('2022-01-01', '2022-12-31')
       .select('LST_Day'))
lst = clipic(lst, roi).toBands()

dem = ee.Image("projects/sat-io/open-datasets/ASTER/GDEM").clip(roi)

# =========================
# Stack & Classify
# =========================
data = ee.Image.cat([
    S2_Monthly,
    S2_Yearly,
    S1_Monthly,
    S1_Yearly,
    lst,
    dem
]).clip(roi)

final_data = data.select(feat_select)

### Features in Astana

In [8]:

#Coordinates
lon = 71.909225
lat = 50.951843

# date_start = '2025-05-01'
# date_end = '2025-10-01'
# Approx 0.1° ≈ 11 km
halfSize = 0.1

rect = ee.Geometry.Rectangle([
    lon - halfSize, lat - halfSize,
    lon + halfSize, lat + halfSize
])

# Create map
m = geemap.Map()
m.set_center(lon, lat, 13)

dataSE_roi = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
          .filterBounds(rect)
          .filterDate(date_start, date_end)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover)))

truecolor = (dataSE_roi
             .median()
             .select(['B4', 'B3', 'B2']))


# Add rectangle layer
# center_point = ee.Geometry.Point([lon, lat])
# m.add_layer(center_point, {'color': 'red'}, 'Center Point')
m.add_layer(truecolor, {'min': 0, 'max': 3000}, 'S2 True Color (median)')
m.add_layer(rect, {'color': 'blue'}, 'Centered Rectangle')
m


Map(center=[50.951843, 71.909225], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=Sear…

In [9]:
# =========================
# Sentinel-2 + Cloud Probability (ROI-first)
# =========================
roi = rect.buffer(2000)

year        = '2025'
date_start  = ee.Date(f'{year}-01-01')
date_end    = ee.Date(f'{year}-12-31')

S2_INPUT_BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

cloudprob = (ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")
             .filterBounds(roi)
             .filterDate(date_start, date_end))

dataSE = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
          .filterBounds(roi)
          .filterDate(date_start, date_end)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover)))

# Join by system:index; keep first cloud prob img as property "Cloud"
idJoin    = ee.Filter.equals(leftField='system:index', rightField='system:index')
innerJoin = ee.Join.saveFirst('Cloud')
dataSE = ee.ImageCollection(innerJoin.apply(primary=dataSE, secondary=cloudprob, condition=idJoin))

# Preprocess and immediately CLIP to ROI
dataSE = (preproS2(dataSE, cloud_prob)
          .select(S2_INPUT_BANDS)
          .map(lambda img: ee.Image(img).clip(roi)))

# Spectral indices (on already-clipped images)
s2Idx = vegetation_indices(dataSE)

# Monthly S2 (median for months 3..10) toBands
months = ee.List.sequence(3, 10)
S2_Monthly = generateS2month(months, s2Idx, year).toBands()

# Yearly S2 (5/50/98) with renamed bands
S2_Yearly = ee.Image(s2Idx.reduce(ee.Reducer.percentile([5, 50, 98])))
S2_Yearly = S2_Yearly.rename(newNames(S2_Yearly, year)).clip(roi)

bands = S2_Monthly.bandNames()
print(len(bands.getInfo()))

bands = S2_Yearly.bandNames()
print(len(bands.getInfo()))


208
78


In [10]:
# =========================
# Sentinel-1 (ROI-first)
# =========================
radar = (ee.ImageCollection("COPERNICUS/S1_GRD")
         .filterBounds(roi)
         .filterDate('2025-01-01', '2026-01-01')
         .filter(ee.Filter.eq('instrumentMode', 'IW'))
         .filter(ee.Filter.listContains('transmitterReceiverPolarisation','VV'))
         .select(['VV','VH']))

radar = preproS1(radar, roi)
radar = s1_features(radar)

# Monthly S1 + Yearly S1
S1_Monthly = generateS1month(radar, '2025-01-01', 12, 1, 'month').toBands()
S1_Yearly  = radar.reduce(ee.Reducer.percentile([5, 50, 98])).clip(roi)

bands = S1_Monthly.bandNames()
print(len(bands.getInfo()))

bands = S1_Yearly.bandNames()
print(len(bands.getInfo()))



63
21


In [11]:
lst = (ee.ImageCollection('MODIS/061/MOD21C3')
       .filterBounds(roi)
       .filterDate('2024-11-01', '2025-10-31')
       .select('LST_Day'))
lst = clipic(lst, roi).toBands()

dem = ee.Image("projects/sat-io/open-datasets/ASTER/GDEM").clip(roi)


In [12]:
img = lst  # your image
old_bandnames = img.bandNames()

# Replace "2024"
new_bandnames = old_bandnames.map(
    lambda b: ee.String(b).replace("2024", "2025")
)

lst_renamed = img.rename(new_bandnames)


In [13]:
year = "2025"

# Replace "2022"
feat_select_updated = feat_select.map(
    lambda s: ee.String(s).replace("2022", year)
)

print("Updated:", feat_select_updated.getInfo())


feat_list = set(feat_select_updated.getInfo())

S1_Monthly_list = set(S1_Monthly.bandNames().getInfo())
S1_Yearly_list = set(S1_Yearly.bandNames().getInfo())
S2_Monthly_list = set(S2_Monthly.bandNames().getInfo())
S2_Yearly_list = set(S2_Yearly.bandNames().getInfo())
lst_list = set(lst_renamed.bandNames().getInfo())

print(list(S1_Monthly_list & feat_list), "\n", len(list(S1_Monthly_list & feat_list)))
print(list(S1_Yearly_list & feat_list), "\n", len(list(S1_Yearly_list & feat_list)))
print(list(S2_Monthly_list & feat_list), "\n", len(list(S2_Monthly_list & feat_list)))
print(list(S2_Yearly_list & feat_list), "\n", len(list(S2_Yearly_list & feat_list)))
print(list(lst_list & feat_list), "\n", len(list(lst_list & feat_list)))


Updated: ['0_2025_B3', '0_2025_B4', '0_2025_B5', '0_2025_BLFEI', '0_2025_LAI', '0_2025_LAIg', '0_2025_LCCI', '0_2025_MSAVI', '0_2025_NDVI', '0_2025_SAVI', '0_2025_SRNIRR', '0_2025_gndvi', '1_2025_B6', '1_2025_B8', '1_2025_BLFEI', '1_2025_DIRESWIR', '1_2025_EVI2', '1_2025_LAI', '1_2025_LAIg', '1_2025_LCCI', '1_2025_MNDWI', '1_2025_MSAVI', '1_2025_NDTI', '1_2025_NDVI', '1_2025_SAVI', '1_2025_SRNIRR', '1_2025_gndvi', '2025_01_01_LST_Day', '2025_02_01_LST_Day', '2025_03_01_LST_Day', '2025_04_01_LST_Day', '2025_05_01_LST_Day', '2025_06_01_LST_Day', '2025_07_01_LST_Day', '2025_08_01_LST_Day', '2025_09_01_LST_Day', '2025_10_01_LST_Day', '2025_11_01_LST_Day', '2025_12_01_LST_Day', '2025_B11_p98', '2025_B12_p50', '2025_B12_p98', '2025_B2_p50', '2025_B3_p50', '2025_B4_p50', '2025_B5_p50', '2025_B6_p98', '2025_B7_p98', '2025_BLFEI_p5', '2025_BLFEI_p50', '2025_BSI_p5', '2025_BSI_p50', '2025_BSI_p98', '2025_DIRESWIR_p50', '2025_EVI2_p5', '2025_EVI2_p98', '2025_LAI_p5', '2025_LAI_p98', '2025_LAIg_p5

In [14]:

data = ee.Image.cat([
    S2_Monthly,
    S2_Yearly,
    S1_Monthly,
    S1_Yearly,
    lst_renamed,
    dem
]).clip(roi)


final_data = data.select(feat_select_updated)

img = final_data  # your image
old_bandnames = img.bandNames()

new_bandnames = old_bandnames.map(
    lambda b: ee.String(b).replace("2025", "2022")
)

final_data_updated = img.rename(new_bandnames)

class_map  = final_data_updated.classify(model).rename('class')

# # Map post-processing / code remap to EU classes and styling
class_map_remap = rename_classes(class_map)
styled_rgb = Visualization(class_map_remap)

In [15]:

m = geemap.Map()
m.set_center(lon, lat, 13)
dataSE_roi = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
          .filterBounds(rect)
          .filterDate(date_start, date_end)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover)))

truecolor = (dataSE_roi
             .median()
             .select(['B4', 'B3', 'B2']))


lon = 71.909225
lat = 50.951843
# 50.94, 71.93

# Add rectangle layer
center_point = ee.Geometry.Point([lon, lat])
m.add_layer(center_point, {'color': 'red'}, 'Center Point')
m.add_layer(truecolor, {'min': 0, 'max': 3000}, 'S2 True Color (median)')
m.add_layer(rect, {'color': 'blue'}, 'Centered Rectangle')
m.add_layer(styled_rgb, {}, 'EU CropMap 2022 (classified)', opacity=1.0)

# Build legend from SLD
items = legend_items_from_sld()   # [{'color':'#hex', 'label':'...'}, ...]
legend_dict = {it['label']: it['color'] for it in items}
m.add_legend(title='EU Crop Classes', legend_dict=legend_dict, position='bottomright')
m.centerObject(roi, 12)
m

Map(center=[50.951810039311766, 71.9092249999676], controls=(WidgetControl(options=['position', 'transparent_b…

#### It doesn't have a pixel at some points

In [16]:
# lon, lat = [71.89230701642431,50.99210643202997] # There exists a class
lon, lat = [71.93,50.94] # dOES NOT exist

point_n = ee.Geometry.Point([lon, lat])
point_e = ee.Geometry.Point([72, 50.94])

samples = class_map.sample(point_n, 10)
print('Number of samples:', samples.size().getInfo())

Number of samples: 0


In [17]:
# Problem is in S2 Monthly

S1_Monthly_samples = S1_Monthly.sample(point_n, 10)
print('S1_Monthly Number of samples:', S1_Monthly_samples.size().getInfo())

S2_Monthly_samples = S2_Monthly.sample(point_n, 10)
print('S2_Monthly Number of samples:', S2_Monthly_samples.size().getInfo())

# Sample S1
s1_fc = S1_Yearly.sample(point_n, 10)
print('S1 sample size:', s1_fc.size().getInfo())

# Sample S2
s2_fc = S2_Yearly.sample(point_n, 10)
print('S2 sample size:', s2_fc.size().getInfo())


S1_Monthly Number of samples: 1
S2_Monthly Number of samples: 0
S1 sample size: 1
S2 sample size: 1


In [18]:
def check_pixel(img):
    img = ee.Image(img)

    # first band name
    band = img.bandNames().get(0)

    # reduce at your point
    pixel = img.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point_n,
        scale=10,
        maxPixels=1e9
    )

    exists = pixel.get(band)     # None if no pixel
    size   = pixel.size()        # number of keys in the dict

    # return FEATURE (valid for Collection.map)
    return ee.Feature(None, {
        'id': img.id(),
        'exists': exists,
        'size': size
    })

# map over the ImageCollection  collection of Features
fc = ee.FeatureCollection(s2Idx.map(check_pixel))

# Pull to client
features = fc.getInfo()['features']


error_list = []

for i, f in enumerate(features):
    props = f['properties']

    if props['exists'] == None:
      error_list.append(props['id'])

    print(f"Image #{i}")
    print("  id    :", props['id'])
    print("  exists:", props['exists'])
    print("------")


print(len(error_list))


Image #0
  id    : 20250108T062139_20250108T062140_T42UXB
  exists: None
------
Image #1
  id    : 20250108T062139_20250108T062140_T42UYB
  exists: None
------
Image #2
  id    : 20250113T062221_20250113T062219_T42UXB
  exists: None
------
Image #3
  id    : 20250113T062221_20250113T062219_T42UYB
  exists: None
------
Image #4
  id    : 20250207T061929_20250207T061931_T42UXB
  exists: None
------
Image #5
  id    : 20250217T061829_20250217T061831_T42UXB
  exists: None
------
Image #6
  id    : 20250217T061829_20250217T061831_T42UYB
  exists: None
------
Image #7
  id    : 20250227T061719_20250227T062333_T42UXB
  exists: None
------
Image #8
  id    : 20250227T061719_20250227T062333_T42UYB
  exists: None
------
Image #9
  id    : 20250304T061801_20250304T061804_T42UXB
  exists: None
------
Image #10
  id    : 20250304T061801_20250304T061804_T42UYB
  exists: None
------
Image #11
  id    : 20250319T061629_20250319T061925_T42UXB
  exists: None
------
Image #12
  id    : 20250319T061629_20

In [19]:
# =========================
# Sentinel-2 + Cloud Probability (ROI-first)
# =========================
roi = rect.buffer(2000)

year        = '2025'
date_start  = ee.Date(f'{year}-01-01')
date_end    = ee.Date(f'{year}-12-31')

S2_INPUT_BANDS = ['B2', 'B3', 'B4', 'B5', 'B6', 'B7', 'B8', 'B8A', 'B11', 'B12']

cloudprob = (ee.ImageCollection("COPERNICUS/S2_CLOUD_PROBABILITY")
             .filterBounds(roi)
             .filterDate(date_start, date_end))

dataSE = (ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
          .filterBounds(roi)
          .filterDate(date_start, date_end)
          .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', cloud_cover)))

# Join by system:index; keep first cloud prob img as property "Cloud"
idJoin    = ee.Filter.equals(leftField='system:index', rightField='system:index')
innerJoin = ee.Join.saveFirst('Cloud')
dataSE = ee.ImageCollection(innerJoin.apply(primary=dataSE, secondary=cloudprob, condition=idJoin))

# Preprocess and immediately CLIP to ROI
dataSE = (preproS2(dataSE, cloud_prob)
          .select(S2_INPUT_BANDS)
          .map(lambda img: ee.Image(img).clip(roi)))

# Spectral indices (on already-clipped images)
s2Idx = vegetation_indices(dataSE)

months = ee.List.sequence(3, 10)
S2_Monthly = generateS2month(months, s2Idx, year).toBands()



In [20]:

# ---- 2. Map over dataSE (NOT s2Idx) ----
fc = ee.FeatureCollection(dataSE.map(check_pixel))

# ---- 3. Bring to client and print image number + size ----
features = fc.getInfo()['features']

for i, f in enumerate(features):
    props = f['properties']

    if props["exists"] == None:
      print(f"Image #{i}")
      print("  id    :", props['id'])
      print("  exists:", props['exists'])
      print("------")

Image #0
  id    : 20250108T062139_20250108T062140_T42UXB
  exists: None
------
Image #1
  id    : 20250108T062139_20250108T062140_T42UYB
  exists: None
------
Image #2
  id    : 20250113T062221_20250113T062219_T42UXB
  exists: None
------
Image #3
  id    : 20250113T062221_20250113T062219_T42UYB
  exists: None
------
Image #4
  id    : 20250207T061929_20250207T061931_T42UXB
  exists: None
------
Image #5
  id    : 20250217T061829_20250217T061831_T42UXB
  exists: None
------
Image #6
  id    : 20250217T061829_20250217T061831_T42UYB
  exists: None
------
Image #7
  id    : 20250227T061719_20250227T062333_T42UXB
  exists: None
------
Image #8
  id    : 20250227T061719_20250227T062333_T42UYB
  exists: None
------
Image #9
  id    : 20250304T061801_20250304T061804_T42UXB
  exists: None
------
Image #10
  id    : 20250304T061801_20250304T061804_T42UYB
  exists: None
------
Image #11
  id    : 20250319T061629_20250319T061925_T42UXB
  exists: None
------
Image #12
  id    : 20250319T061629_20

In [21]:
data_list = dataSE.toList(dataSE.size())

img1 = ee.Image(data_list.get(0))   # None esxisting image
img2 = ee.Image(data_list.get(30))   # Cloudmasked image
img3 = ee.Image(data_list.get(31))   # Full image

print("1st image ID :", img1.get('system:index').getInfo())
print("2nd image ID :", img2.get('system:index').getInfo())
print("5th image ID :", img3.get('system:index').getInfo())

true_img = img1

true_time = ee.Date(true_img.get('system:time_start'))
true_id = true_img.get('system:index').getInfo()

# Print time (UTC)
print("First dataSE image time (UTC):",
      true_time.format('YYYY-MM-dd HH:mm:ss').getInfo())

s2_raw = ee.Image('COPERNICUS/S2_SR_HARMONIZED/' + true_id)


Map = geemap.Map()
Map.centerObject(point_n, 12)

vis = {'bands': ['B4', 'B3', 'B2'], 'min': 0, 'max': 3000}

Map.addLayer(s2_raw, vis, f"S2 raw at {true_time.format('YYYY-MM-dd').getInfo()}")
Map.addLayer(img1, vis, "Image 1 (1st)")
Map.addLayer(img2, vis, "Image 2 (2nd)")
Map.addLayer(img3, vis, "Image 5 (5th)")

Map

1st image ID : 20250108T062139_20250108T062140_T42UXB
2nd image ID : 20250622T061651_20250622T061823_T42UYB
5th image ID : 20250702T061651_20250702T062054_T42UXB
First dataSE image time (UTC): 2025-01-08 06:25:04


Map(center=[50.94, 71.93], controls=(WidgetControl(options=['position', 'transparent_bg'], widget=SearchDataGU…

In [24]:
def img_to_feature(img):
    img = ee.Image(img)

    # any first band
    band = img.bandNames().get(0)

    # attempt to read pixel at point_n
    pixel = img.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=point_n,
        scale=10,
        maxPixels=1e9
    )

    exists = pixel.get(band)  # None = no data at pixel
    month  = ee.Date(img.get('system:time_start')).get('month')

    return ee.Feature(None, {
        'month': month,
        'exists': exists
    })

fc = ee.FeatureCollection(dataSE.map(img_to_feature))

all_hist = (fc
            .reduceColumns(
                ee.Reducer.frequencyHistogram(),
                ['month']
            )
            .get('histogram'))

missing_hist = (fc
                .filter(ee.Filter.eq('exists', None))
                .reduceColumns(
                    ee.Reducer.frequencyHistogram(),
                    ['month']
                )
                .get('histogram'))

all_hist_py     = all_hist.getInfo() or {}
missing_hist_py = missing_hist.getInfo() or {}

months = sorted(all_hist_py.keys(), key=int)

for m in months:
    total   = all_hist_py.get(m, 0)
    missing = missing_hist_py.get(m, 0)
    print(f"Month {m}: total images = {total}, missing pixel = {missing}")


Month 1: total images = 4, missing pixel = 4
Month 2: total images = 5, missing pixel = 5
Month 3: total images = 6, missing pixel = 6
Month 4: total images = 3, missing pixel = 1
Month 5: total images = 3, missing pixel = 0
Month 6: total images = 10, missing pixel = 2
Month 7: total images = 11, missing pixel = 0
Month 8: total images = 8, missing pixel = 0
Month 9: total images = 8, missing pixel = 1
Month 10: total images = 7, missing pixel = 0
Month 11: total images = 3, missing pixel = 3


# Dump

In [ ]:
lon, lat = [71.89230701642431,50.99210643202997] # There exists a class
# lon, lat = [71.931185548966,50.93840538167604] # dOES NOT exist

point = ee.Geometry.Point([lon, lat])

samples = class_map.sample(point, 10)
print('Number of samples:', samples.size().getInfo())

first_feat = samples.first()
value = first_feat.get('class').getInfo()

print('Pixel class:', value)


In [ ]:
# lon, lat = [71.89230701642431,50.99210643202997] # There exists a class
lon, lat = [71.931185548966,50.93840538167604] # dOES NOT exist

point = ee.Geometry.Point([lon, lat])

pixel = data.select(feat_select).sample(point, 10).first()
print(data.sample(point, 10).size().getInfo())
print(pixel.getInfo())


In [ ]:
def check_img(name, img):
    fc = img.sample(point, 10)
    size = fc.size().getInfo()
    print(f'{name}: samples =', size)
    if size > 0:
        print(f'{name} first feature:', fc.first().getInfo())

check_img('S2_Monthly', S2_Monthly)
check_img('S2_Yearly',  S2_Yearly)
check_img('S1_Monthly', S1_Monthly)
check_img('S1_Yearly',  S1_Yearly)
check_img('LST',        lst)
check_img('DEM',        dem)



### Astana fields issue
they do not have enogh features,

In [ ]:
# Find non-existing feature names
feature_list = feat_select.getInfo()
bands_list = data.bandNames().getInfo()

intersection = list(set(feature_list) & set(bands_list))

print("Intersection:", intersection)
print("Number of intersecting bands:", len(intersection))

diff = list(set(feature_list) - set(intersection))
print("Elements in feature_list but not in intersection:", diff)
print("Count:", len(diff))


In [ ]:
print('S1 raw size:', radar.size().getInfo())

In [ ]:
def s1_diagnose(roi, start='2022-01-01', end='2023-01-01'):
    geom = roi.geometry() if hasattr(roi, 'geometry') else roi
    ic0 = (ee.ImageCollection('COPERNICUS/S1_GRD')
           .filterBounds(geom)
           .filterDate(start, end))
    print('S1 total (any mode/pol):', ic0.size().getInfo())
    if ic0.size().getInfo() == 0:
        # try relaxing date and ROI a bit
        ic_relaxed = (ee.ImageCollection('COPERNICUS/S1_GRD')
                      .filterBounds(geom.buffer(20000))  # +20 km
                      .filterDate('2022-01-01', '2024-01-01'))
        print('Relaxed search size:', ic_relaxed.size().getInfo())
        return

    # What instrument modes?
    modes = ic0.aggregate_array('instrumentMode').distinct().getInfo()
    print('instrumentMode:', modes)

    # What polarisations appear?
    pols = (ic0.aggregate_array('transmitterReceiverPolarisation')
             .distinct().getInfo())
    print('transmitterReceiverPolarisation (distinct arrays):', pols)

    # Asc/desc passes:
    passes = ic0.aggregate_array('orbitProperties_pass').distinct().getInfo()
    print('orbitProperties_pass:', passes)

    # Show one example image props
    info = ic0.first().toDictionary().getInfo()
    keys = ['id','instrumentMode','transmitterReceiverPolarisation',
            'orbitProperties_pass','relativeOrbitNumber_start']
    print({k: info.get(k) for k in keys})

# call it
s1_diagnose(roi)


In [ ]:

def get_s1_resilient(
    roi,
    start='2022-01-01',
    end='2023-01-01',
    require_iw=True,
    try_dual_pol_first=True,
    fall_back_vv_only=False,
    buffer_seq_m=(0, 5000, 10000, 20000, 40000),
    year_slack_seq=(0, 1, 2)
):
    """
    Try to find S1 over ROI by expanding buffer and date window.
    Returns (ic, meta) where ic is ee.ImageCollection (either dual-pol or VV-only per options),
    meta is a dict describing what was used.
    """
    base_start = ee.Date(start)
    base_end   = ee.Date(end)
    geom0 = roi.geometry() if hasattr(roi, 'geometry') else roi

    def _filtered_ic(g, s, e):
        ic = ee.ImageCollection('COPERNICUS/S1_GRD').filterBounds(g).filterDate(s, e)
        if require_iw:
            ic = ic.filter(ee.Filter.eq('instrumentMode', 'IW'))
        return ic

    def _dual(ic):
        return (ic.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                 .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VH'))
                 .select(['VV', 'VH']))

    def _vv_only(ic):
        return (ic.filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
                 .select(['VV']))

    # Try combos of (buffer, year_slack)
    for y_slack in year_slack_seq:
        s = base_start.advance(-y_slack, 'year')
        e = base_end.advance( y_slack, 'year')
        for buf in buffer_seq_m:
            g = geom0 if buf == 0 else geom0.buffer(buf)
            ic0 = _filtered_ic(g, s, e)

            if try_dual_pol_first:
                ic_dual = _dual(ic0)
                n_dual = ic_dual.size().getInfo()
                if n_dual > 0:
                    return ic_dual, {'mode': 'VV,VH', 'count': n_dual, 'buffer_m': buf, 'start': s.format().getInfo(), 'end': e.format().getInfo()}

                if fall_back_vv_only:
                    ic_vv = _vv_only(ic0)
                    n_vv = ic_vv.size().getInfo()
                    if n_vv > 0:
                        return ic_vv, {'mode': 'VV', 'count': n_vv, 'buffer_m': buf, 'start': s.format().getInfo(), 'end': e.format().getInfo()}
            else:
                ic_vv = _vv_only(ic0)
                n_vv = ic_vv.size().getInfo()
                if n_vv > 0:
                    return ic_vv, {'mode': 'VV', 'count': n_vv, 'buffer_m': buf, 'start': s.format().getInfo(), 'end': e.format().getInfo()}

    raise RuntimeError("No Sentinel-1 images found even after expanding buffer/date. "
                       "Relax constraints further or inspect area/mode.")


In [ ]:
radar, meta = get_s1_resilient(
    roi,
    start='2022-01-01', end='2023-01-01',
    require_iw=True,
    try_dual_pol_first=True,
    fall_back_vv_only=False,  # set True only if you accept VV-only features
)
print(meta)


In [ ]:
# Build features on the (possibly buffered) collection
radar_feat = s1_features(preproS1(radar, roi))  # your helpers

# Monthly stack (then flip names like 'VH_2' -> '2_VH' to match training)
S1_Monthly_raw = generateS1month(radar_feat, '2022-01-01', 12, 1, 'month').toBands()
S1_Monthly = enforce_index_first_for_monthly(S1_Monthly_raw)  # from earlier message

# Yearly percentiles
S1_Yearly = radar_feat.reduce(ee.Reducer.percentile([5, 50, 98]))

# IMPORTANT: clip the stacks back to your exact roi
S1_Monthly = S1_Monthly.clip(roi)
S1_Yearly  = S1_Yearly.clip(roi)


In [ ]:
def s1_months_fill_empty(radar_feat, start='2022-01-01'):
    """Ensure 12 monthly windows exist with expected bands."""
    bands_example = ee.Image(radar_feat.first()).bandNames()
    # windows: [Jan...Dec]
    ic = generateS1month(radar_feat, start, 12, 1, 'month')  # 12 windows
    # For any empty window, make a constant-0 image with same bands
    def _ensure(img):
        img = ee.Image(img)
        cond = img.bandNames().size().gt(0)
        filler = ee.Image(0).rename(bands_example).toFloat()
        return ee.Image(ee.Algorithms.If(cond, img, filler))
    return ee.ImageCollection(ic.map(_ensure)).toBands()

S1_Monthly_raw = s1_months_fill_empty(radar_feat, '2022-01-01')
S1_Monthly     = enforce_index_first_for_monthly(S1_Monthly_raw).clip(roi)


In [ ]:
# Count images over the buffered search (meta['buffer_m']) area
coverage = radar.count()
# Or monthly availability:
monthly_count = generateS1month(radar, '2022-01-01', 12, 1, 'month').count().toBands()

m = geemap.Map()
m.center_object(roi, 9)
m.add_layer(coverage, {'min':0, 'max':50}, 'S1 count')
m.add_layer(roi, {'color':'white'}, 'ROI')
m


## Fix

In [ ]:
def mask_edge(image):
  edge = image.lt(-30.0)
  masked_image = image.mask().And(edge.Not())
  return image.updateMask(masked_image)


img_vv = (
    ee.ImageCollection('COPERNICUS/S1_GRD')
    .filter(ee.Filter.listContains('transmitterReceiverPolarisation', 'VV'))
    .filter(ee.Filter.eq('instrumentMode', 'IW'))
    .select('VV')
    .map(mask_edge)
)

desc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'DESCENDING'))
asc = img_vv.filter(ee.Filter.eq('orbitProperties_pass', 'ASCENDING'))

spring = ee.Filter.date('2015-03-01', '2015-04-20')
late_spring = ee.Filter.date('2015-04-21', '2015-06-10')
summer = ee.Filter.date('2015-06-11', '2015-08-31')

desc_change = ee.Image.cat(
    desc.filter(spring).mean(),
    desc.filter(late_spring).mean(),
    desc.filter(summer).mean(),
)

asc_change = ee.Image.cat(
    asc.filter(spring).mean(),
    asc.filter(late_spring).mean(),
    asc.filter(summer).mean(),
)

m = geemap.Map()
m.set_center(lon, lat, 13)
m.add_layer(asc_change, {'min': -25, 'max': 5}, 'Multi-T Mean ASC', True)
m.add_layer(desc_change, {'min': -25, 'max': 5}, 'Multi-T Mean DESC', True)
m

In [ ]:
def mask_s2_clouds(image):
  """Masks clouds in a Sentinel-2 image using the QA band.

  Args:
      image (ee.Image): A Sentinel-2 image.

  Returns:
      ee.Image: A cloud-masked Sentinel-2 image.
  """
  qa = image.select('QA60')

  # Bits 10 and 11 are clouds and cirrus, respectively.
  cloud_bit_mask = 1 << 10
  cirrus_bit_mask = 1 << 11

  # Both flags should be set to zero, indicating clear conditions.
  mask = (
      qa.bitwiseAnd(cloud_bit_mask)
      .eq(0)
      .And(qa.bitwiseAnd(cirrus_bit_mask).eq(0))
  )

  return image.updateMask(mask).divide(10000)


dataset = (
    ee.ImageCollection('COPERNICUS/S2_SR_HARMONIZED')
    .filterDate('2020-01-01', '2020-01-30')
    # Pre-filter to get less cloudy granules.
    .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE', 20))
    .map(mask_s2_clouds)
)

visualization = {
    'min': 0.0,
    'max': 0.3,
    'bands': ['B4', 'B3', 'B2'],
}

m = geemap.Map()
m.set_center(lon, lat, 12)
m.add_layer(dataset.mean(), visualization, 'RGB')
m